In [2]:
import numpy as np

# Define the potential functions for each clique. Here we use simple functions for demonstration.
def psi1(x1, x2):
    return np.exp(-abs(x1 - x2))

def psi2(x2, x3):
    return np.exp(-abs(x2 - x3))

# List all configurations of X1, X2, X3
configurations = [(x1, x2, x3) for x1 in [0, 1] for x2 in [0, 1] for x3 in [0, 1]]

# Compute the partition function Z
Z = 0
for config in configurations:
    x1, x2, x3 = config
    prob = psi1(x1, x2) * psi2(x2, x3)
    Z += prob

# Compute the joint probability distribution
joint_probabilities = {}
for config in configurations:
    x1, x2, x3 = config
    prob = psi1(x1, x2) * psi2(x2, x3) / Z
    joint_probabilities[config] = prob

# Output the results
prob_total = 0
for config, prob in joint_probabilities.items():
    prob_total += prob
    print(f"Probability of {config}: {prob:.4f}")

print(prob_total)

Probability of (0, 0, 0): 0.2672
Probability of (0, 0, 1): 0.0983
Probability of (0, 1, 0): 0.0362
Probability of (0, 1, 1): 0.0983
Probability of (1, 0, 0): 0.0983
Probability of (1, 0, 1): 0.0362
Probability of (1, 1, 0): 0.0983
Probability of (1, 1, 1): 0.2672
1.0


In [4]:
import numpy as np

# Define the potential functions for each clique
def psi1(x1, x2):
    return np.exp(-abs(x1 - x2))

def psi2(x2, x3):
    return np.exp(-abs(x2 - x3))

# List all configurations of X1, X2, X3
configurations = [(x1, x2, x3) for x1 in [0, 1] for x2 in [0, 1] for x3 in [0, 1]]

# Compute the partition function Z
Z = 0
joint_probabilities = {}
for config in configurations:
    x1, x2, x3 = config
    prob = psi1(x1, x2) * psi2(x2, x3)
    joint_probabilities[config] = prob
    Z += prob

# Compute marginals
marginals = {
    'X1': np.zeros(2),
    'X2': np.zeros(2),
    'X3': np.zeros(2),
    'X1_X2': np.zeros((2, 2)),
    'X2_X3': np.zeros((2, 2))
}

for (x1, x2, x3), joint_prob in joint_probabilities.items():
    joint_prob /= Z  # Normalize the joint probability
    marginals['X1'][x1] += joint_prob
    marginals['X2'][x2] += joint_prob
    marginals['X3'][x3] += joint_prob
    marginals['X1_X2'][x1, x2] += joint_prob
    marginals['X2_X3'][x2, x3] += joint_prob

# Output the results
for var, dist in marginals.items():
    if dist.ndim == 1:
        for i in range(len(dist)):
            print(f"Marginal P({var}={i}): {dist[i]:.4f}")
    else:
        for i in range(dist.shape[0]):
            for j in range(dist.shape[1]):
                print(f"Marginal P({var}=({i}, {j})): {dist[i, j]:.4f}")

Marginal P(X1=0): 0.5000
Marginal P(X1=1): 0.5000
Marginal P(X2=0): 0.5000
Marginal P(X2=1): 0.5000
Marginal P(X3=0): 0.5000
Marginal P(X3=1): 0.5000
Marginal P(X1_X2=(0, 0)): 0.3655
Marginal P(X1_X2=(0, 1)): 0.1345
Marginal P(X1_X2=(1, 0)): 0.1345
Marginal P(X1_X2=(1, 1)): 0.3655
Marginal P(X2_X3=(0, 0)): 0.3655
Marginal P(X2_X3=(0, 1)): 0.1345
Marginal P(X2_X3=(1, 0)): 0.1345
Marginal P(X2_X3=(1, 1)): 0.3655


In [ ]:
import torch
    
# 00, 01, 10, 11
A1B = torch.tensor([[0, 0], [0.5, 1]])

A2C = torch.tensor([[0, 0], [0.7, 1]])

BD = torch.tensor([[0.5, 0.1], [0.1, 1]])

CE = torch.tensor([[0.5, 0.1], [0.1, 1]])

# 000, 010, 001, 111, 100, 110, 101, 111
DEF = torch.tensor([[0.5, 0, 0, 0], [0, 0, 0, 0.5]])

In [5]:
import torch

# Factors for edges A1-B, A2-C, B-D, C-E
A1B = torch.tensor([[0, 0], [0.5, 1]])
A2C = torch.tensor([[0, 0], [0.7, 1]])
BD = torch.tensor([[0.5, 0.1], [0.1, 1]])
CE = torch.tensor([[0.5, 0.1], [0.1, 1]])

# Factor for the triple D-E-F
DEF = torch.tensor([[0.5, 0, 0, 0], [0, 0, 0, 0.5]])

def normalize(tensor):
    return tensor / tensor.sum()

# Next, we perform marginalization for each variable by summing over the factor tables
# This will involve marginalizing over the joint probability tables one variable at a time

# Marginalize over A1-B to get marginals for A1 and B
marginal_A1 = normalize(A1B.sum(dim=1))
marginal_B = normalize(A1B.sum(dim=0))

# Marginalize over A2-C to get marginals for A2 and C
marginal_A2 = normalize(A2C.sum(dim=1))
marginal_C = normalize(A2C.sum(dim=0))

# Marginalize over B-D to get marginal for D (B is already marginalized)
marginal_BD = torch.matmul(marginal_B.view(1, -1), BD)
marginal_D = normalize(marginal_BD.sum(dim=0))

# Marginalize over C-E to get marginal for E (C is already marginalized)
marginal_CE = torch.matmul(marginal_C.view(1, -1), CE)
marginal_E = normalize(marginal_CE.sum(dim=0))

# For the factor DEF, we will perform marginalization over D, E to get marginal for F
# and then marginalize over F to get the marginals for D and E
marginal_DEF = DEF.view(2, 2, 2)  # Reshape to 2x2x2 tensor for easier marginalization
marginal_F = normalize(marginal_DEF.sum(dim=0).sum(dim=0))
marginal_DE = normalize(marginal_DEF.sum(dim=2))

# Now marginalize over D to get marginal for E and over E to get marginal for D
marginal_D_from_DEF = normalize(marginal_DE.sum(dim=1))
marginal_E_from_DEF = normalize(marginal_DE.sum(dim=0))

# Combine the marginals for D and E from their respective factors
# This is done by multiplying the marginals obtained from BD/CE and DEF factors and then normalizing
marginal_D_combined = normalize(marginal_D * marginal_D_from_DEF)
marginal_E_combined = normalize(marginal_E * marginal_E_from_DEF)

# Marginals for variables A1, A2, B, C, D, E, F
marginals = {
    "A1": marginal_A1,
    "A2": marginal_A2,
    "B": marginal_B,
    "C": marginal_C,
    "D": marginal_D_combined,
    "E": marginal_E_combined,
    "F": marginal_F
}

# Display the results
marginals

{'A1': tensor([0., 1.]),
 'A2': tensor([0., 1.]),
 'B': tensor([0.3333, 0.6667]),
 'C': tensor([0.4118, 0.5882]),
 'D': tensor([0.2500, 0.7500]),
 'E': tensor([0.2961, 0.7039]),
 'F': tensor([0.5000, 0.5000])}